In [108]:
import pandas as pd
import numpy as np
import time
from sklearn.model_selection import train_test_split
import sys
import turicreate as tc
sys.path.append("..")
import json

In [109]:
customers = pd.read_csv('./data/recommend_1.csv') 
transactions = pd.read_csv('./data/trx_data.csv')
customers.head()

,customerId
0,0
1,1
2,2
3,3
4,4


In [110]:

print(transactions.shape)
transactions.head()
transactions['products'] = transactions['products'].apply(lambda x: [int(i) for i in x.split("|")])
transactions.head(2).set_index('customerId')['products'].apply(pd.Series).reset_index()

(8, 2)


,customerId,0,1,2
0,0,2,3,4
1,1,1,3,4


In [111]:
data = pd.melt(transactions.set_index('customerId')['products'].apply(pd.Series).reset_index(), 
             id_vars=['customerId'],
             value_name='products') \
    .dropna().drop(['variable'], axis=1) \
    .groupby(['customerId', 'products']) \
    .agg({'products': 'count'}) \
    .rename(columns={'products': 'purchase_count'}) \
    .reset_index() \
    .rename(columns={'products': 'productId'})


data['productId']=data['productId'].astype(np.int64)


In [112]:
data.head()



,customerId,productId,purchase_count
0,0,2,1
1,0,3,1
2,0,4,1
3,1,1,1
4,1,3,1


In [113]:
def create_data_dummy(data):
    data_dummy = data.copy()
    data_dummy['purchase_dummy'] = 1
    return data_dummy
data_dummy = create_data_dummy(data)

In [114]:
df_matrix = pd.pivot_table(data, values='purchase_count', index='customerId', columns='productId')

In [115]:
df_matrix.head()


productId,1,2,3,4,5,6,7,8
customerId,,,,,,,,
0,NaN,1.0,1.0,1.0,NaN,NaN,NaN,NaN
1,1.0,NaN,1.0,1.0,1.0,NaN,NaN,NaN
2,NaN,1.0,2.0,NaN,NaN,NaN,NaN,NaN
3,NaN,1.0,1.0,NaN,NaN,1.0,NaN,1.0
4,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN


In [116]:
#df_matrix_norm = (df_matrix-df_matrix.min())/(df_matrix.max()-df_matrix.min())
df_matrix_norm=df_matrix

In [117]:
# create a table for input to the modeling  
d = df_matrix_norm.reset_index() 
d.index.names = ['scaled_purchase_freq'] 
data_norm = pd.melt(d, id_vars=['customerId'], value_name='scaled_purchase_freq').dropna()
print(data_norm.shape)
data_norm.head()

(14, 3)


,customerId,productId,scaled_purchase_freq
1,1,1,1.0
5,0,2,1.0
7,2,2,1.0
8,3,2,1.0
10,0,3,1.0


In [118]:
def split_data(data):
    '''
    Splits dataset into training and test set.
    
    Args:
        data (pandas.DataFrame)
        
    Returns
        train_data (tc.SFrame)
        test_data (tc.SFrame)
    '''
    train, test = train_test_split(data, test_size = .2)
    train_data = tc.SFrame(train)
    test_data = tc.SFrame(test)
    return train_data, test_data



In [119]:
train_data, test_data = split_data(data)
train_data_dummy, test_data_dummy = split_data(data_dummy)
train_data_norm, test_data_norm = split_data(data_norm)

In [120]:
train_data

customerId,productId,purchase_count
1,3,1
3,6,1
4,7,1
3,2,1
0,2,1
1,1,1
3,3,1
2,3,2
1,4,1
0,3,1


In [121]:
train_data

customerId,productId,purchase_count
1,3,1
3,6,1
4,7,1
3,2,1
0,2,1
1,1,1
3,3,1
2,3,2
1,4,1
0,3,1


In [122]:
# constant variables to define field names include:
user_id = 'customerId'
item_id = 'productId'
users_to_recommend = list(customers[user_id])
n_rec = 10 # number of items to recommend
n_display = 30 # to display the first few rows in an output dataset

In [123]:
def model(train_data,name,user_id,item_id,target,users_to_recommend,n_rec,n_display):
    if name=='popularity':
        model=tc.popularity_recommender.create(train_data,user_id=user_id,item_id=item_id,target=target)
    elif name=='cosine':
        model = tc.item_similarity_recommender.create(train_data, user_id=user_id, item_id=item_id, target=target,similarity_type='cosine')
    elif name == 'pearson':
        model = tc.item_similarity_recommender.create(train_data,user_id=user_id,  item_id=item_id,  target=target,similarity_type='pearson')
    recom = model.recommend(users=users_to_recommend, k=n_rec)
    recom.print_rows(n_display)
    return model
        

In [124]:
name = 'popularity'
target = 'purchase_count'
popularity = model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 11 observations with 5 users and 6 items.

Data prepared in: 0.008851s

11 observations to process; with 6 unique items.

+------------+-----------+-------+------+
| customerId | productId | score | rank |
+------------+-----------+-------+------+
|     0      |     6     |  1.0  |  1   |
|     0      |     7     |  1.0  |  2   |
|     0      |     1     |  1.0  |  3   |
|     0      |     4     |  1.0  |  4   |
|     1      |     6     |  1.0  |  1   |
|     1      |     7     |  1.0  |  2   |
|     1      |     2     |  1.0  |  3   |
|     2      |     6     |  1.0  |  1   |
|     2      |     7     |  1.0  |  2   |
|     2      |     1     |  1.0  |  3   |
|     2      |     4     |  1.0  |  4   |
|     3      |     7     |  1.0  |  1   |
|     3      |     1     |  1.0  |  2   |
|     3      |     4     |  1.0  |  3   |
|     4      |     3     |  1.25 |  1   |
|     4      |     6     |  1.0  |  2   |
|     4      |     2     |  1.0  |  3   |
|     4      |     1     |  1.0  |  4   |
|     4      |     4     |  1.0  |  5   |
+------------+-----------+-------+------+
[19 rows x 4 columns]



In [125]:
name = 'popularity'
target = 'purchase_dummy'
pop_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Warning: Ignoring columns purchase_count;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 11 observations with 4 users and 8 items.

Data prepared in: 0.005623s

11 observations to process; with 8 unique items.

+------------+-----------+-------+------+
| customerId | productId | score | rank |
+------------+-----------+-------+------+
|     0      |     5     |  1.0  |  1   |
|     0      |     6     |  1.0  |  2   |
|     0      |     7     |  1.0  |  3   |
|     0      |     8     |  1.0  |  4   |
|     0      |     1     |  1.0  |  5   |
|     1      |     2     |  1.0  |  1   |
|     1      |     3     |  1.0  |  2   |
|     1      |     6     |  1.0  |  3   |
|     1      |     7     |  1.0  |  4   |
|     1      |     8     |  1.0  |  5   |
|     2      |     5     |  1.0  |  1   |
|     2      |     4     |  1.0  |  2   |
|     2      |     2     |  1.0  |  3   |
|     2      |     3     |  1.0  |  4   |
|     2      |     6     |  1.0  |  5   |
|     2      |     7     |  1.0  |  6   |
|     2      |     8     |  1.0  |  7   |
|     2      |     1     |  1.0  |  8   |
|     3      |     5     |  1.0  |  1   |
|     3      |     4     |  1.0  |  2   |
|     3      |     7     |  1.0  |

In [126]:
name = 'popularity'
target = 'scaled_purchase_freq'
pop_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 11 observations with 5 users and 7 items.

Data prepared in: 0.002805s

11 observations to process; with 7 unique items.

+------------+-----------+-------+------+
| customerId | productId | score | rank |
+------------+-----------+-------+------+
|     0      |     3     |  1.5  |  1   |
|     0      |     6     |  1.0  |  2   |
|     0      |     7     |  1.0  |  3   |
|     0      |     8     |  1.0  |  4   |
|     0      |     1     |  1.0  |  5   |
|     1      |     2     |  1.0  |  1   |
|     1      |     6     |  1.0  |  2   |
|     1      |     7     |  1.0  |  3   |
|     1      |     8     |  1.0  |  4   |
|     2      |     4     |  1.0  |  1   |
|     2      |     6     |  1.0  |  2   |
|     2      |     7     |  1.0  |  3   |
|     2      |     8     |  1.0  |  4   |
|     2      |     1     |  1.0  |  5   |
|     3      |     3     |  1.5  |  1   |
|     3      |     4     |  1.0  |  2   |
|     3      |     7     |  1.0  |  3   |
|     3      |     1     |  1.0  |  4   |
|     4      |     3     |  1.5  |  1   |
|     4      |     4     |  1.0  |  2   |
|     4      |     2     |  1.0  |

In [127]:
name = 'cosine'
target = 'purchase_count'
cos = model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 11 observations with 5 users and 6 items.

Data prepared in: 0.003333s

Training model from provided data.

Gathering per-item and per-user statistics.

...

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.024694s

+------------+-----------+----------------------+------+
| customerId | productId |        score         | rank |
+------------+-----------+----------------------+------+
|     0      |     6     | 0.47765737771987915  |  1   |
|     0      |     1     | 0.18898224830627441  |  2   |
|     0      |     4     | 0.18898224830627441  |  3   |
|     0      |     7     |         0.0          |  4   |
|     1      |     2     |  0.2909571925799052  |  1   |
|     1      |     6     | 0.12598816553751627  |  2   |
|     1      |     7     |         0.0          |  3   |
|     2      |     6     |  0.6666396260261536  |  1   |
|     2      |     1     | 0.37796449661254883  |  2   |
|     2      |     4     | 0.37796449661254883  |  3   |
|     2      |     7     |         0.0          |  4   |
|     3      |     1     | 0.12598816553751627  |  1   |
|     3      |     4     | 0.12598816553751627  |  2   |
|     3      |     7     |         0.0          |  3   |
|     4      |     2     | 0.03

In [128]:
name = 'cosine' 
target = 'scaled_purchase_freq' 
cos_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

Preparing data set.

Data has 11 observations with 5 users and 7 items.

Data prepared in: 0.002443s

Training model from provided data.

Gathering per-item and per-user statistics.

...

+--------------------------------+------------+

| 6.41ms                         | 100        |

+------------+-----------+----------------------+------+
| customerId | productId |        score         | rank |
+------------+-----------+----------------------+------+
|     0      |     3     |  0.4163127839565277  |  1   |
|     0      |     1     |  0.3535533845424652  |  2   |
|     0      |     6     | 0.28867512941360474  |  3   |
|     0      |     8     | 0.28867512941360474  |  4   |
|     0      |     7     |         0.0          |  5   |
|     1      |     2     |  0.3082153598467509  |  1   |
|     1      |     6     |         0.0          |  2   |
|     1      |     7     |         0.0          |  3   |
|     1      |     8     |         0.0          |  4   |
|     2      |     4     |  0.5203519463539124  |  1   |
|     2      |     1     |  0.4472135901451111  |  2   |
|     2      |     6     | 0.28867512941360474  |  3   |
|     2      |     8     | 0.28867512941360474  |  4   |
|     2      |     7     |         0.0          |  5   |
|     3      |     3     |  0.1

+--------------------------------+------------+

Setting up lookup tables.

In [129]:
name = 'cosine'
target = 'purchase_dummy'
cos_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)



Processing data in one pass using dense lookup tables.

+-------------------------------------+------------------+-----------------+

| Elapsed Time (Constructing Lookups) | Total % Complete | Items Processed |

+-------------------------------------+------------------+-----------------+

| 6.709ms                             | 14.25            | 1               |

...

+-------------------------------------+------------------+-----------------+

+------------+-----------+---------------------+------+
| customerId | productId |        score        | rank |
+------------+-----------+---------------------+------+
|     0      |     6     | 0.47140451272328693 |  1   |
|     0      |     8     | 0.47140451272328693 |  2   |
|     0      |     5     | 0.23570225636164346 |  3   |
|     0      |     1     | 0.23570225636164346 |  4   |
|     0      |     7     |         0.0         |  5   |
|     1      |     2     | 0.16666666666666666 |  1   |
|     1      |     3     | 0.16666666666666666 |  2   |
|     1      |     6     |         0.0         |  3   |
|     1      |     7     |         0.0         |  4   |
|     1      |     8     |         0.0         |  5   |
|     2      |     5     |         0.0         |  1   |
|     2      |     4     |         0.0         |  2   |
|     2      |     2     |         0.0         |  3   |
|     2      |     3     |         0.0         |  4   |
|     2      |     6     |         0.0         |

| Elapsed Time (Constructing Lookups) | Total % Complete | Items Processed |

+-------------------------------------+------------------+-----------------+

| 1.465ms                             | 25               | 2               |

In [130]:
name = 'pearson'
target = 'purchase_count'
pear = model(train_data, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

| 7.755ms                             | 100              | 8               |

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.012676s

...

Setting up lookup tables.

+------------+-----------+-------+------+
| customerId | productId | score | rank |
+------------+-----------+-------+------+
|     0      |     6     |  0.0  |  1   |
|     0      |     7     |  0.0  |  2   |
|     0      |     1     |  0.0  |  3   |
|     0      |     4     |  0.0  |  4   |
|     1      |     6     |  0.0  |  1   |
|     1      |     7     |  0.0  |  2   |
|     1      |     2     |  0.0  |  3   |
|     2      |     6     |  0.0  |  1   |
|     2      |     7     |  0.0  |  2   |
|     2      |     1     |  0.0  |  3   |
|     2      |     4     |  0.0  |  4   |
|     3      |     7     |  0.0  |  1   |
|     3      |     1     |  0.0  |  2   |
|     3      |     4     |  0.0  |  3   |
|     4      |     3     |  0.25 |  1   |
|     4      |     6     |  0.0  |  2   |
|     4      |     2     |  0.0  |  3   |
|     4      |     1     |  0.0  |  4   |
|     4      |     4     |  0.0  |  5   |
+------------+-----------+-------+------+
[19 rows x 4 columns]



Processing data in one pass using dense lookup tables.

+-------------------------------------+------------------+-----------------+

| Elapsed Time (Constructing Lookups) | Total % Complete | Items Processed |

In [131]:
name = 'pearson'
target = 'purchase_dummy'
pear_dummy = model(train_data_dummy, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)

+-------------------------------------+------------------+-----------------+

| 818us                               | 33.25            | 2               |

| 2.908ms                             | 100              | 6               |

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

...

| 12.172ms                       | 100        |

+--------------------------------+------------+

+------------+-----------+-------+------+
| customerId | productId | score | rank |
+------------+-----------+-------+------+
|     0      |     5     |  0.0  |  1   |
|     0      |     6     |  0.0  |  2   |
|     0      |     7     |  0.0  |  3   |
|     0      |     8     |  0.0  |  4   |
|     0      |     1     |  0.0  |  5   |
|     1      |     2     |  0.0  |  1   |
|     1      |     3     |  0.0  |  2   |
|     1      |     6     |  0.0  |  3   |
|     1      |     7     |  0.0  |  4   |
|     1      |     8     |  0.0  |  5   |
|     2      |     5     |  0.0  |  1   |
|     2      |     4     |  0.0  |  2   |
|     2      |     2     |  0.0  |  3   |
|     2      |     3     |  0.0  |  4   |
|     2      |     6     |  0.0  |  5   |
|     2      |     7     |  0.0  |  6   |
|     2      |     8     |  0.0  |  7   |
|     2      |     1     |  0.0  |  8   |
|     3      |     5     |  0.0  |  1   |
|     3      |     4     |  0.0  |  2   |
|     3      |     7     |  0.0  |

Setting up lookup tables.

Processing data in one pass using dense lookup tables.

In [132]:
name = 'pearson'
target = 'scaled_purchase_freq'
pear_norm = model(train_data_norm, name, user_id, item_id, target, users_to_recommend, n_rec, n_display)



+-------------------------------------+------------------+-----------------+

| Elapsed Time (Constructing Lookups) | Total % Complete | Items Processed |

+-------------------------------------+------------------+-----------------+

| 17.744ms                            | 1.5              | 0               |

| 18.344ms                            | 100              | 8               |

...

Training model from provided data.

Gathering per-item and per-user statistics.

+------------+-----------+-------+------+
| customerId | productId | score | rank |
+------------+-----------+-------+------+
|     0      |     3     |  0.5  |  1   |
|     0      |     6     |  0.0  |  2   |
|     0      |     7     |  0.0  |  3   |
|     0      |     8     |  0.0  |  4   |
|     0      |     1     |  0.0  |  5   |
|     1      |     2     |  0.0  |  1   |
|     1      |     6     |  0.0  |  2   |
|     1      |     7     |  0.0  |  3   |
|     1      |     8     |  0.0  |  4   |
|     2      |     4     |  0.0  |  1   |
|     2      |     6     |  0.0  |  2   |
|     2      |     7     |  0.0  |  3   |
|     2      |     8     |  0.0  |  4   |
|     2      |     1     |  0.0  |  5   |
|     3      |     3     |  0.5  |  1   |
|     3      |     4     |  0.0  |  2   |
|     3      |     7     |  0.0  |  3   |
|     3      |     1     |  0.0  |  4   |
|     4      |     3     |  0.5  |  1   |
|     4      |     4     |  0.0  |  2   |
|     4      |     2     |  0.0  |

+--------------------------------+------------+

| Elapsed Time (Item Statistics) | % Complete |

In [133]:
models_w_counts = [popularity, cos, pear]
models_w_dummy = [pop_dummy, cos_dummy, pear_dummy]
models_w_norm = [pop_norm, cos_norm, pear_norm]
names_w_counts = ['Popularity Model on Purchase Counts', 'Cosine Similarity on Purchase Counts', 'Pearson Similarity on Purchase Counts']
names_w_dummy = ['Popularity Model on Purchase Dummy', 'Cosine Similarity on Purchase Dummy', 'Pearson Similarity on Purchase Dummy']
names_w_norm = ['Popularity Model on Scaled Purchase Counts', 'Cosine Similarity on Scaled Purchase Counts', 'Pearson Similarity on Scaled Purchase Counts']

+--------------------------------+------------+

| 11.26ms                        | 100        |

+--------------------------------+------------+

In [134]:
eval_counts = tc.recommender.util.compare_models(test_data, models_w_counts, model_names=names_w_counts)


Setting up lookup tables.

Processing data in one pass using dense lookup tables.

+-------------------------------------+------------------+-----------------+

| Elapsed Time (Constructing Lookups) | Total % Complete | Items Processed |

+-------------------------------------+------------------+-----------------+

...

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.014171s

PROGRESS: Evaluate model Popularity Model on Purchase Counts

Precision and recall summary statistics by cutoff
+--------+---------------------+---------------------+
| cutoff |    mean_precision   |     mean_recall     |
+--------+---------------------+---------------------+
|   1    |         0.0         |         0.0         |
|   2    |         0.0         |         0.0         |
|   3    |         0.0         |         0.0         |
|   4    | 0.08333333333333333 |  0.3333333333333333 |
|   5    | 0.08333333333333334 | 0.33333333333333337 |
|   6    | 0.08333333333333334 | 0.33333333333333337 |
|   7    | 0.08333333333333334 | 0.33333333333333337 |
|   8    | 0.08333333333333334 | 0.33333333333333337 |
|   9    | 0.08333333333333334 | 0.33333333333333337 |
|   10   | 0.08333333333333334 | 0.33333333333333337 |
+--------+---------------------+---------------------+
[10 rows x 3 columns]


Overall RMSE: 0.07422696190252048

Per User RMSE (best)
+------------+------+-------+
| custom

In [135]:
eval_dummy = tc.recommender.util.compare_models(test_data_dummy, models_w_dummy, model_names=names_w_dummy)

PROGRESS: Evaluate model Popularity Model on Purchase Dummy

Precision and recall summary statistics by cutoff
+--------+---------------------+-------------+
| cutoff |    mean_precision   | mean_recall |
+--------+---------------------+-------------+
|   1    |         0.0         |     0.0     |
|   2    |         0.25        |     0.5     |
|   3    |  0.3333333333333333 |     0.75    |
|   4    |        0.375        |     1.0     |
|   5    | 0.30000000000000004 |     1.0     |
|   6    | 0.26666666666666666 |     1.0     |
|   7    | 0.24285714285714285 |     1.0     |
|   8    |        0.225        |     1.0     |
|   9    |        0.225        |     1.0     |
|   10   |        0.225        |     1.0     |
+--------+---------------------+-------------+
[10 rows x 3 columns]


Overall RMSE: 0.0

Per User RMSE (best)
+------------+------+-------+
| customerId | rmse | count |
+------------+------+-------+
|     2      | 0.0  |   2   |
+------------+------+-------+
[1 rows x 3 colum

In [136]:
eval_norm = tc.recommender.util.compare_models(test_data_norm, models_w_norm, model_names=names_w_norm)

PROGRESS: Evaluate model Popularity Model on Scaled Purchase Counts

Precision and recall summary statistics by cutoff
+--------+---------------------+--------------------+
| cutoff |    mean_precision   |    mean_recall     |
+--------+---------------------+--------------------+
|   1    |  0.6666666666666667 | 0.6666666666666667 |
|   2    | 0.33333333333333337 | 0.6666666666666667 |
|   3    |  0.2222222222222222 | 0.6666666666666667 |
|   4    | 0.16666666666666669 | 0.6666666666666667 |
|   5    |         0.15        | 0.6666666666666666 |
|   6    |         0.15        | 0.6666666666666666 |
|   7    |         0.15        | 0.6666666666666666 |
|   8    |         0.15        | 0.6666666666666666 |
|   9    |         0.15        | 0.6666666666666666 |
|   10   |         0.15        | 0.6666666666666666 |
+--------+---------------------+--------------------+
[10 rows x 3 columns]


Overall RMSE: 0.4116084153698826

Per User RMSE (best)
+------------+---------------------+-------+
|

In [137]:
final_model = tc.item_similarity_recommender.create(tc.SFrame(data_norm),user_id=user_id, item_id=item_id,target='scaled_purchase_freq', similarity_type='cosine')

Preparing data set.

Data has 14 observations with 5 users and 8 items.

Data prepared in: 0.0077s

Training model from provided data.

Gathering per-item and per-user statistics.

+--------------------------------+------------+

| Elapsed Time (Item Statistics) | % Complete |

+--------------------------------+------------+

In [138]:
recom = final_model.recommend(users=users_to_recommend, k=n_rec)
recom.print_rows(n_display)

| 1.387ms                        | 100        |

+--------------------------------+------------+

Setting up lookup tables.

Processing data in one pass using dense lookup tables.

+-------------------------------------+------------------+-----------------+

...

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 0.022586s

+------------+-----------+----------------------+------+
| customerId | productId |        score         | rank |
+------------+-----------+----------------------+------+
|     0      |     1     | 0.36169042189915973  |  1   |
|     0      |     5     | 0.36169042189915973  |  2   |
|     0      |     6     | 0.31843825181325275  |  3   |
|     0      |     8     | 0.31843825181325275  |  4   |
|     0      |     7     |         0.0          |  5   |
|     1      |     2     | 0.32027997076511383  |  1   |
|     1      |     6     | 0.09449112415313721  |  2   |
|     1      |     8     | 0.09449112415313721  |  3   |
|     1      |     7     |         0.0          |  4   |
|     2      |     4     |  0.7386466264724731  |  1   |
|     2      |     6     |  0.6666396260261536  |  2   |
|     2      |     8     |  0.6666396260261536  |  3   |
|     2      |     1     | 0.37796449661254883  |  4   |
|     2      |     5     | 0.37796449661254883  |  5   |
|     2      |     7     |     

In [139]:
for i in recom:
    print(i)
    

{'customerId': 0, 'productId': 1, 'score': 0.36169042189915973, 'rank': 1}
{'customerId': 0, 'productId': 5, 'score': 0.36169042189915973, 'rank': 2}
{'customerId': 0, 'productId': 6, 'score': 0.31843825181325275, 'rank': 3}
{'customerId': 0, 'productId': 8, 'score': 0.31843825181325275, 'rank': 4}
{'customerId': 0, 'productId': 7, 'score': 0.0, 'rank': 5}
{'customerId': 1, 'productId': 2, 'score': 0.32027997076511383, 'rank': 1}
{'customerId': 1, 'productId': 6, 'score': 0.09449112415313721, 'rank': 2}
{'customerId': 1, 'productId': 8, 'score': 0.09449112415313721, 'rank': 3}
{'customerId': 1, 'productId': 7, 'score': 0.0, 'rank': 4}
{'customerId': 2, 'productId': 4, 'score': 0.7386466264724731, 'rank': 1}
{'customerId': 2, 'productId': 6, 'score': 0.6666396260261536, 'rank': 2}
{'customerId': 2, 'productId': 8, 'score': 0.6666396260261536, 'rank': 3}
{'customerId': 2, 'productId': 1, 'score': 0.37796449661254883, 'rank': 4}
{'customerId': 2, 'productId': 5, 'score': 0.377964496612548

In [141]:
json_output=[]
for i in recom:
    json_output.append(i)

In [142]:
json_output

[{'customerId': 0, 'productId': 1, 'score': 0.36169042189915973, 'rank': 1},
 {'customerId': 0, 'productId': 5, 'score': 0.36169042189915973, 'rank': 2},
 {'customerId': 0, 'productId': 6, 'score': 0.31843825181325275, 'rank': 3},
 {'customerId': 0, 'productId': 8, 'score': 0.31843825181325275, 'rank': 4},
 {'customerId': 0, 'productId': 7, 'score': 0.0, 'rank': 5},
 {'customerId': 1, 'productId': 2, 'score': 0.32027997076511383, 'rank': 1},
 {'customerId': 1, 'productId': 6, 'score': 0.09449112415313721, 'rank': 2},
 {'customerId': 1, 'productId': 8, 'score': 0.09449112415313721, 'rank': 3},
 {'customerId': 1, 'productId': 7, 'score': 0.0, 'rank': 4},
 {'customerId': 2, 'productId': 4, 'score': 0.7386466264724731, 'rank': 1},
 {'customerId': 2, 'productId': 6, 'score': 0.6666396260261536, 'rank': 2},
 {'customerId': 2, 'productId': 8, 'score': 0.6666396260261536, 'rank': 3},
 {'customerId': 2, 'productId': 1, 'score': 0.37796449661254883, 'rank': 4},
 {'customerId': 2, 'productId': 5,

In [149]:
final_output=json.dumps(json_output)

In [150]:
final_output

'[{"customerId": 0, "productId": 1, "score": 0.36169042189915973, "rank": 1}, {"customerId": 0, "productId": 5, "score": 0.36169042189915973, "rank": 2}, {"customerId": 0, "productId": 6, "score": 0.31843825181325275, "rank": 3}, {"customerId": 0, "productId": 8, "score": 0.31843825181325275, "rank": 4}, {"customerId": 0, "productId": 7, "score": 0.0, "rank": 5}, {"customerId": 1, "productId": 2, "score": 0.32027997076511383, "rank": 1}, {"customerId": 1, "productId": 6, "score": 0.09449112415313721, "rank": 2}, {"customerId": 1, "productId": 8, "score": 0.09449112415313721, "rank": 3}, {"customerId": 1, "productId": 7, "score": 0.0, "rank": 4}, {"customerId": 2, "productId": 4, "score": 0.7386466264724731, "rank": 1}, {"customerId": 2, "productId": 6, "score": 0.6666396260261536, "rank": 2}, {"customerId": 2, "productId": 8, "score": 0.6666396260261536, "rank": 3}, {"customerId": 2, "productId": 1, "score": 0.37796449661254883, "rank": 4}, {"customerId": 2, "productId": 5, "score": 0.